# Problem set 2: Machine Learning 2024.
## Alumnos: Ara Francisco, Corradi Valentin, Di Placido Pedro.

In [1]:
import numpy as np
import pandas as pd

## General Instructions

El principal objetivo es construir un modelo que prediga la pobreza de los hogares. Un hogar es clasificado según la siguiente función indicadora:

$$
poor = I(Inc < Pl)
$$

Donde I indica si los ingresos de la familia están por debajo de cierto umbral.

Entonces, podemos seguir 2 caminos para hacer esta clasificación:

1 - Tomarlo como un problema de clasificación, donde se prediga 0 si no es pobre y 1 si lo es

2 - Predecir el ingreso, y luego fijarse si esta por debajo o por encima del umbral de pobreza.




## Introduction 



In [12]:
import zipfile

zip_path = "mlunlp-2024-ps-2.zip"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    # Listar todos los archivos dentro del ZIP
    print(zip_ref.namelist())

['mlunlp-2024-ps-2/', '__MACOSX/._mlunlp-2024-ps-2', 'mlunlp-2024-ps-2/train_hogares.csv', '__MACOSX/mlunlp-2024-ps-2/._train_hogares.csv', 'mlunlp-2024-ps-2/ddi-documentation-spanish-608.pdf', '__MACOSX/mlunlp-2024-ps-2/._ddi-documentation-spanish-608.pdf', 'mlunlp-2024-ps-2/test_personas.csv', '__MACOSX/mlunlp-2024-ps-2/._test_personas.csv', 'mlunlp-2024-ps-2/train_personas.csv', '__MACOSX/mlunlp-2024-ps-2/._train_personas.csv', 'mlunlp-2024-ps-2/test_hogares.csv', '__MACOSX/mlunlp-2024-ps-2/._test_hogares.csv', 'mlunlp-2024-ps-2/sample_submission.csv', '__MACOSX/mlunlp-2024-ps-2/._sample_submission.csv']


In [13]:
import zipfile
import pandas as pd

# Ruta al archivo ZIP
zip_path = "mlunlp-2024-ps-2.zip"

# Leer archivos directamente desde el ZIP
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    # Cargar los datos usando las rutas correctas
    with zip_ref.open('mlunlp-2024-ps-2/train_hogares.csv') as train_hogares_file:
        train_hogares = pd.read_csv(train_hogares_file)
    with zip_ref.open('mlunlp-2024-ps-2/train_personas.csv') as train_personas_file:
        train_personas = pd.read_csv(train_personas_file)
    with zip_ref.open('mlunlp-2024-ps-2/test_hogares.csv') as test_hogares_file:
        test_hogares = pd.read_csv(test_hogares_file)
    with zip_ref.open('mlunlp-2024-ps-2/test_personas.csv') as test_personas_file:
        test_personas = pd.read_csv(test_personas_file)
    with zip_ref.open('mlunlp-2024-ps-2/sample_submission.csv') as sample_submission_file:
        sample_submission = pd.read_csv(sample_submission_file)

# Inspeccionar los datos
print("Datos de train_hogares:")
print(train_hogares.head())
print("Datos de train_personas:")
print(train_personas.head())


Datos de train_hogares:
                         id  Clase   Dominio  P5000  P5010  P5090  P5100  \
0  8d0193e328305a042001a35f      1  MEDELLIN      4      2      1    NaN   
1  cf7491d7f2cc9c216bd009e7      1  MEDELLIN      5      4      3    NaN   
2  d90a57f64d2a84dbacbed2a5      1  MEDELLIN      4      2      1    NaN   
3  db34e387a94a783188ab3a33      1  MEDELLIN      3      1      1    NaN   
4  97bbef785824746ecab09c6f      1  MEDELLIN      4      3      3    NaN   

      P5130     P5140  Nper  ...       Ingpcug             Li            Lp  \
0  400000.0       NaN     5  ...  5.816661e+05  122809.534158  289878.24715   
1       NaN  250000.0     4  ...  1.612500e+05  122809.534158  289878.24715   
2  700000.0       NaN     3  ...  2.000000e+06  122809.534158  289878.24715   
3  450000.0       NaN     1  ...  2.450000e+06  122809.534158  289878.24715   
4       NaN  500000.0     5  ...  2.520000e+05  122809.534158  289878.24715   

   Pobre  Indigente  Npobres  Nindigentes   

In [16]:
# Matcheamos lo individual con el hogar. previamente le agregamos los sufijos _p o _h para saber su base original.

# Personas
train_personas = train_personas.rename(columns=lambda col: f"{col}_p" if col != "id" else col)
train_hogares = train_hogares.rename(columns=lambda col: f"{col}_h" if col != "id" else col)
train_all = train_personas.merge(train_hogares, on="id", how="inner")  

#Hogares
test_personas = test_personas.rename(columns=lambda col: f"{col}_p" if col != "id" else col)
test_hogares = test_hogares.rename(columns=lambda col: f"{col}_h" if col != "id" else col)
test_all = test_personas.merge(test_hogares, on="id", how="inner")

print(f'Dimensión train_all: {train_all.shape}, Dimensión test_all: {test_all.shape}')

Dimensión train_all: (543109, 157), Dimensión test_all: (219644, 78)


In [ ]:
print(train_hogares.shape)
print(train_personas.shape)
print(test_hogares.shape)
print(test_personas.shape)

(164960, 23)
(543109, 135)
(66168, 16)
(219644, 63)


Vemos que la dimensión de los df difiere. La muestra de testeo no tiene las variables relacionadas al ingreso o el indicador de pobreza/indigencia. Esto no debiera afectarnos ya que es la variable que buscamos predecir. Recordemos que un metodo a utilizar es categórico, donde en base a los predictores se clasifica directamente en pobre o no pobre, y el otro es "indirecto", donde primero se predice el ingreso y en base a eso se hace luego la clasificación.

In [ ]:
#print(train_hogares.columns)
#print(test_hogares.columns)

print(f'Las variables que difieren en los hogares son: \n{set(train_hogares.columns) - set(test_hogares.columns)}')

#print(train_personas.columns)
#print(test_personas.columns)

print(f'Las variables que difieren en las personas son: \n {set(train_personas.columns) - set(test_personas.columns)}')

Index(['id', 'Clase', 'Dominio', 'P5000', 'P5010', 'P5090', 'P5100', 'P5130',
       'P5140', 'Nper', 'Npersug', 'Ingtotug', 'Ingtotugarr', 'Ingpcug', 'Li',
       'Lp', 'Pobre', 'Indigente', 'Npobres', 'Nindigentes', 'Fex_c', 'Depto',
       'Fex_dpto'],
      dtype='object')
Las variables que difieren en los hogares son: 
{'Ingtotug', 'Pobre', 'Npobres', 'Ingtotugarr', 'Nindigentes', 'Ingpcug', 'Indigente'}
Las variables que difieren en las personas son: 
 {'Iof6es', 'Iof3h', 'Impaes', 'P6585s1a1', 'Iof3i', 'P6590s1', 'P7510s6a1', 'P7500s1a1', 'Cclasnr11', 'Cclasnr6', 'P7472s1', 'P7070', 'Iof6', 'Isaes', 'Iof3hes', 'P6585s4a2', 'P6585s2a1', 'Cclasnr5', 'Ingtotes', 'P6580s1', 'P6580s2', 'Iees', 'P7500s3a1', 'Cclasnr7', 'Iof2es', 'P6630s4a1', 'Estrato1', 'Cclasnr4', 'P6545s2', 'P6630s6a1', 'P6510s1', 'P7500s2a1', 'P6600s1', 'Cclasnr3', 'P7510s3a1', 'P7422s1', 'P6510s2', 'Iof3ies', 'P6585s3a1', 'Imdies', 'Cclasnr2', 'P6585s2a2', 'P7140s2', 'P6500', 'P7140s1', 'P6610s1', 'Ie', 'P6620s1',

In [30]:
# print(train_hogares.columns)
print(F'test per: {test_personas.columns}')

test per: Index(['id', 'Orden_p', 'Clase_p', 'Dominio_p', 'P6020_p', 'P6040_p',
       'P6050_p', 'P6090_p', 'P6100_p', 'P6210_p', 'P6210s1_p', 'P6240_p',
       'Oficio_p', 'P6426_p', 'P6430_p', 'P6510_p', 'P6545_p', 'P6580_p',
       'P6585s1_p', 'P6585s2_p', 'P6585s3_p', 'P6585s4_p', 'P6590_p',
       'P6600_p', 'P6610_p', 'P6620_p', 'P6630s1_p', 'P6630s2_p', 'P6630s3_p',
       'P6630s4_p', 'P6630s6_p', 'P6800_p', 'P6870_p', 'P6920_p', 'P7040_p',
       'P7045_p', 'P7050_p', 'P7090_p', 'P7110_p', 'P7120_p', 'P7150_p',
       'P7160_p', 'P7310_p', 'P7350_p', 'P7422_p', 'P7472_p', 'P7495_p',
       'P7500s2_p', 'P7500s3_p', 'P7505_p', 'P7510s1_p', 'P7510s2_p',
       'P7510s3_p', 'P7510s5_p', 'P7510s6_p', 'P7510s7_p', 'Pet_p', 'Oc_p',
       'Des_p', 'Ina_p', 'Fex_c_p', 'Depto_p', 'Fex_dpto_p'],
      dtype='object')


In [32]:
train_hogares['Pobre_h'].mean()


np.float64(0.20019398642095054)